# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'03-11-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'03-11-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-03-12 05:26:50,33.93911,67.709953,55917,2451,49499.0,3967.0,Afghanistan,143.640850,4.383282
1,NaN,NaN,NaN,Albania,2021-03-12 05:26:50,41.15330,20.168300,115442,2002,78259.0,35181.0,Albania,4011.467093,1.734204
2,NaN,NaN,NaN,Algeria,2021-03-12 05:26:50,28.03390,1.659600,114851,3028,79556.0,32267.0,Algeria,261.911672,2.636459
3,NaN,NaN,NaN,Andorra,2021-03-12 05:26:50,42.50630,1.521800,11130,112,10708.0,310.0,Andorra,14404.969909,1.006289
4,NaN,NaN,NaN,Angola,2021-03-12 05:26:50,-11.20270,17.873900,21205,517,19827.0,861.0,Angola,64.519038,2.438104


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55759,55770,55775,55827,55840,55847,55876,55876,55894,55917
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,108823,109674,110521,111301,112078,112897,113580,114209,114840,115442
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,113430,113593,113761,113948,114104,114234,114382,114543,114681,114851


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2446,2446,2446,2449,2449,2449,2451,2451,2451,2451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1835,1856,1876,1897,1918,1939,1956,1969,1986,2002
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2991,2996,3002,3007,3010,3013,3018,3022,3026,3028


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,49347,49359,49362,49362,49369,49378,49402,49409,49499,49499
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,71173,72076,72853,73610,74352,75190,75887,76784,77498,78259
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,78377,78524,78672,78824,78946,79064,79187,79319,79428,79556


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6303,6313,6324,6333,6344,6347,6364,6371,6400,6409
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19790,19856,19873,19890,19915,19935,19942,19962,20012,20044
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2124,2129,2136,2139,2138,2139,2143,2147,2161,2171


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,91,91,92,92,92,92,92,92,93,95
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,284,285,289,289,289,289,289,290,293,294
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,51,51,51,51,51,51,51,51,51,52


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-03-12 05:26:50,32.539527,-86.644082,6409,95,NaN,NaN,"Autauga, Alabama, US",11471.477922,1.482291
689,1075.0,Lamar,Alabama,US,2021-03-12 05:26:50,33.779950,-88.096680,1351,33,NaN,NaN,"Lamar, Alabama, US",9786.309308,2.442635
690,1077.0,Lauderdale,Alabama,US,2021-03-12 05:26:50,34.901719,-87.656247,8662,214,NaN,NaN,"Lauderdale, Alabama, US",9341.198546,2.470561


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,29285001,530727,0.0
India,11308846,158306,10953303.0
Brazil,11277717,272889,9983656.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,29285001,530727,0.0,28754274.0,2021-03-12 05:26:50,37.936303,-91.379001
India,11308846,158306,10953303.0,197237.0,2021-03-12 05:26:50,23.088275,81.806127
Brazil,11277717,272889,9983656.0,1021172.0,2021-03-12 05:26:50,-12.669522,-48.480493
Russia,4311893,89224,3914866.0,307803.0,2021-03-12 05:26:50,54.546312,62.120860
United Kingdom,4254714,125403,11944.0,4117367.0,2021-03-12 05:26:50,28.164647,-33.966498


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3613719,55066,0.0
Texas,2715993,46007,0.0
Florida,1962651,32040,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3613719,55066,0.0,3558653.0,2021-03-12 05:26:50,37.843962,-120.728594
Texas,2715993,46007,0.0,2669986.0,2021-03-12 05:26:50,31.660643,-98.653069
Florida,1962651,32040,0.0,1930611.0,2021-03-12 05:26:50,28.940755,-82.700744
New York,1720199,48690,0.0,1671509.0,2021-03-12 05:26:50,42.544151,-75.474183
Illinois,1204323,23123,0.0,1181200.0,2021-03-12 05:26:50,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1208024,22304,0.0
Arizona,Maricopa,519285,9393,0.0
Illinois,Cook,481101,9494,0.0
Florida,Miami-Dade,423714,5619,0.0
Texas,Harris,362716,5401,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1208024,22304,0.0,1185720.0,2021-03-12 05:26:50,34.308284,-118.228241,6037.0
Arizona,Maricopa,519285,9393,0.0,509892.0,2021-03-12 05:26:50,33.348359,-112.491815,4013.0
Illinois,Cook,481101,9494,0.0,471607.0,2021-03-12 05:26:50,41.841448,-87.816588,17031.0
Florida,Miami-Dade,423714,5619,0.0,418095.0,2021-03-12 05:26:50,25.611236,-80.551706,12086.0
Texas,Harris,362716,5401,0.0,357315.0,2021-03-12 05:26:50,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,55876,114209,114543,11089,21114,862,2162001,175198,29074,479391,...,4241858,65527,80247,3,143321,2526,200382,2586,82897,36321
2021-03-10,55894,114840,114681,11130,21161,882,2169694,175538,29090,481919,...,4247879,66484,80268,3,143321,2529,202378,2627,83333,36341
2021-03-11,55917,115442,114851,11130,21205,882,2177898,176286,29102,484916,...,4254714,67717,80392,3,144277,2533,203669,2667,83913,36377


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,2451,1969,3022,112,516,22,53252,3232,909,8757,...,125032,666,622,0,1399,35,2170,654,1137,1489
2021-03-10,2451,1986,3026,112,516,23,53359,3237,909,8776,...,125222,678,622,0,1399,35,2193,661,1140,1489
2021-03-11,2451,2002,3028,112,517,23,53493,3239,909,8798,...,125403,683,622,0,1415,35,2211,667,1148,1492


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,49409,76784,79319,10685,19677,455,1956591,165096,22936,447041,...,11868,56399,78939,1,135389,2004,178121,1451,79162,33919
2021-03-10,49499,77498,79428,10708,19761,489,1961640,165259,22945,449053,...,11923,57134,78994,1,135389,2004,179810,1452,79556,33953
2021-03-11,49499,78259,79556,10708,19827,489,1968462,165441,22949,450862,...,11944,57773,79077,1,136269,2048,181452,1464,80027,33972


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6303,6313,6324,6333,6344,6347,6364,6371,6400,6409
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1331,1337,1337,1339,1339,1339,1339,1340,1349,1351
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8601,8610,8612,8618,8628,8630,8631,8636,8651,8662
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2836,2846,2849,2853,2857,2862,2863,2865,2864,2869
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,14980,15021,15039,15057,15070,15083,15084,15087,15108,15117


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-09,500616,59712,0,827800,325383,3607891,438025,288145,88679,49,...,785242,2704712,375669,16286,2744,588129,346403,133856,622444,54972
2021-03-10,501398,59863,0,828630,325700,3611295,439265,288657,88891,49,...,786597,2710278,376327,16371,2755,589375,347131,134158,623150,55014
2021-03-11,502263,60099,0,830655,326040,3613719,440220,289392,89680,49,...,788109,2715993,376973,16500,2761,590625,347884,134496,623801,55112


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-03-09        6371   19962    2147  2466   6255    1177   2020   13977   
2021-03-10        6400   20012    2161  2469   6260    1180   2022   13989   
2021-03-11        6409   20044    2171  2474   6274    1181   2035   14007   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-09         3435     1784  ...    2622    597     3038      681   
2021-03-10         3438     1786  ...    2622    597     3039      681   
2021-03-11         3438     1785  ...    2623    597     3042      681   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-09           3859  3395  2083          0      888    623  
2021-03-10           3861  3404  2084          0      888    623  
2021-03-11           3870  3418  2087          0      888    631  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-09,10186,305,0,16326,5357,54628,5997,7739,1482,0,...,11588,45582,1990,211,25,9790,5077,2326,7134,691
2021-03-10,10222,305,0,16404,5382,54890,6003,7752,1492,0,...,11606,45796,1992,211,25,9849,5100,2330,7150,691
2021-03-11,10274,305,0,16461,5410,55066,6021,7761,1494,0,...,11623,46007,2015,211,25,9902,5107,2338,7160,691


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-03-09          92     290      51   58    128      38     66     296   
2021-03-10          93     293      51   58    129      39     66     296   
2021-03-11          95     294      52   58    129      39     66     297   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-03-09          112       41  ...      29     11       30        7   
2021-03-10          113       42  ...      29     11       30        7   
2021-03-11          113       42  ...      29     11       30        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-09             36     9    12          0       26      5  
2021-03-10             36     9    12          0       26      5  
2021-03-11             36     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,0.000000,0.005538,0.001408,0.001807,0.000284,0.016509,0.003391,0.001040,0.000447,0.005055,...,0.001386,0.012782,0.000349,0.0,0.003831,0.000792,0.009207,0.016110,0.002928,0.000882
2021-03-10,0.000322,0.005525,0.001205,0.003697,0.002226,0.023202,0.003558,0.001941,0.000550,0.005273,...,0.001419,0.014605,0.000262,0.0,0.000000,0.001188,0.009961,0.015855,0.005260,0.000551
2021-03-11,0.000411,0.005242,0.001482,0.000000,0.002079,0.000000,0.003781,0.004261,0.000413,0.006219,...,0.001609,0.018546,0.001545,0.0,0.006670,0.001582,0.006379,0.015226,0.006960,0.000991


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,0.0,0.006646,0.001325,0.0,0.001942,0.047619,0.002466,0.002171,0.0,0.002863,...,0.001851,0.012158,0.0,NaN,0.005751,0.0,0.006027,0.001531,0.004417,0.001345
2021-03-10,0.0,0.008634,0.001324,0.0,0.000000,0.045455,0.002009,0.001547,0.0,0.002170,...,0.001520,0.018018,0.0,NaN,0.000000,0.0,0.010599,0.010703,0.002639,0.000000
2021-03-11,0.0,0.008056,0.000661,0.0,0.001938,0.000000,0.002511,0.000618,0.0,0.002507,...,0.001445,0.007375,0.0,NaN,0.011437,0.0,0.008208,0.009077,0.007018,0.002015


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,0.000142,0.011820,0.001667,0.002251,0.001017,0.109756,0.002588,0.000861,0.000218,0.004514,...,0.001266,0.013678,0.000304,0.0,0.004079,0.043750,0.009144,0.002764,0.005602,0.001388
2021-03-10,0.001822,0.009299,0.001374,0.002153,0.004269,0.074725,0.002581,0.000987,0.000392,0.004501,...,0.004634,0.013032,0.000697,0.0,0.000000,0.000000,0.009482,0.000689,0.004977,0.001002
2021-03-11,0.000000,0.009820,0.001612,0.000000,0.003340,0.000000,0.003478,0.001101,0.000174,0.004028,...,0.001761,0.011184,0.001051,0.0,0.006500,0.021956,0.009132,0.008264,0.005920,0.000560


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-09,0.001048,0.001476,NaN,0.000681,0.001329,0.001530,0.001917,0.002606,0.001310,0.0,...,0.001707,0.002086,0.001490,0.005371,0.000000,0.002620,0.001944,0.001714,0.000968,0.001257
2021-03-10,0.001562,0.002529,NaN,0.001003,0.000974,0.000943,0.002831,0.001777,0.002391,0.0,...,0.001726,0.002058,0.001752,0.005219,0.004009,0.002119,0.002102,0.002256,0.001134,0.000764
2021-03-11,0.001725,0.003942,NaN,0.002444,0.001044,0.000671,0.002174,0.002546,0.008876,0.0,...,0.001922,0.002109,0.001717,0.007880,0.002178,0.002121,0.002169,0.002519,0.001045,0.001781


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-09      0.001100  0.001003  0.001867  0.000812 -0.000160  0.000000   
2021-03-10      0.004552  0.002505  0.006521  0.001217  0.000799  0.002549   
2021-03-11      0.001406  0.001599  0.004627  0.002025  0.002236  0.000847   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-09      0.001487  0.000644  0.000291  0.000561  ... -0.000381   
2021-03-10      0.000990  0.000859  0.000873  0.001121  ...  0.000000   
2021-03-11      0.006429  0.001287  0.000000 -0.000560  ...  0.000381   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-03-09     -0.001672  0.000329      0.0   0.001557 -0.000589  0.003372   
2021-03-10      0.000000  0.000329      0.0   0.000518  0.002651  0.000480   
2021-03-11      0.000000  0.000987      0.0   0.002331  0.004113  0.001440   

Province_State                                
Admin2         Unassigned Washakie    Weston  
2021-03-09            NaN      0.0  0.000000  
2021-03-10            NaN      0.0  0.000000  
2021-03-11            NaN      0.0  0.012841  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-09,0.003745,0.0,NaN,-0.000122,0.002620,0.005744,0.001169,0.001812,0.004746,NaN,...,0.002769,0.003766,0.006576,0.014423,0.0,0.011050,0.002765,0.000430,0.003940,0.013196
2021-03-10,0.003534,0.0,NaN,0.004778,0.004667,0.004796,0.001001,0.001680,0.006748,NaN,...,0.001553,0.004695,0.001005,0.000000,0.0,0.006027,0.004530,0.001720,0.002243,0.000000
2021-03-11,0.005087,0.0,NaN,0.003475,0.005203,0.003206,0.002999,0.001161,0.001340,NaN,...,0.001465,0.004607,0.011546,0.000000,0.0,0.005381,0.001373,0.003433,0.001399,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                      \
Admin2           Autauga   Baldwin   Barbour Bibb    Blount   Bullock Butler   
2021-03-09      0.000000  0.003460  0.000000  0.0  0.000000  0.055556    0.0   
2021-03-10      0.010870  0.010345  0.000000  0.0  0.007812  0.026316    0.0   
2021-03-11      0.021505  0.003413  0.019608  0.0  0.000000  0.000000    0.0   

Province_State                               ...   Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...      Park Platte Sheridan   
2021-03-09      0.000000  0.009009  0.02500  ...  0.074074    0.0      0.0   
2021-03-10      0.000000  0.008929  0.02439  ...  0.000000    0.0      0.0   
2021-03-11      0.003378  0.000000  0.00000  ...  0.000000    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-03-09          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-03-10          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-03-11          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,0.000195,0.006098,0.001353,0.002041,0.000812,0.014671,0.002816,0.001638,0.000437,0.004785,...,0.001317,0.013437,0.000451,0.500,0.003514,0.002503,0.009376,0.018383,0.003786,0.000677
2021-03-10,0.000259,0.005811,0.001279,0.002869,0.001519,0.018936,0.003187,0.001790,0.000494,0.005029,...,0.001368,0.014021,0.000356,0.250,0.001757,0.001845,0.009669,0.017119,0.004523,0.000614
2021-03-11,0.000335,0.005527,0.001381,0.001435,0.001799,0.009468,0.003484,0.003025,0.000453,0.005624,...,0.001489,0.016283,0.000951,0.125,0.004214,0.001714,0.008024,0.016173,0.005741,0.000802


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,0.000247,0.008268,0.001371,0.000143,0.002121,0.028247,0.002692,0.001664,1.166070e-25,0.002581,...,0.001355,0.011338,1.199774e-11,NaN,0.005287,1.013723e-58,0.006742,0.002196,0.005261,0.001176
2021-03-10,0.000124,0.008451,0.001347,0.000071,0.001061,0.036851,0.002351,0.001605,5.830348e-26,0.002375,...,0.001437,0.014678,5.998869e-12,NaN,0.002643,5.068617e-59,0.008671,0.006450,0.003950,0.000588
2021-03-11,0.000062,0.008254,0.001004,0.000036,0.001499,0.018425,0.002431,0.001112,2.915174e-26,0.002441,...,0.001441,0.011026,2.999435e-12,NaN,0.007040,2.534309e-59,0.008439,0.007763,0.005484,0.001301


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-03-09,0.000229,0.010944,0.001624,0.002485,0.000932,0.076752,0.002764,0.000965,0.000149,0.004408,...,0.001016,0.013711,0.000481,0.0,0.003737,0.022128,0.008380,0.002120,0.006685,0.001521
2021-03-10,0.001025,0.010121,0.001499,0.002319,0.002600,0.075738,0.002672,0.000976,0.000271,0.004454,...,0.002825,0.013372,0.000589,0.0,0.001868,0.011064,0.008931,0.001404,0.005831,0.001262
2021-03-11,0.000513,0.009970,0.001555,0.001159,0.002970,0.037869,0.003075,0.001039,0.000223,0.004241,...,0.002293,0.012278,0.000820,0.0,0.004184,0.016510,0.009032,0.004834,0.005876,0.000911


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-09,0.000961,0.002333,NaN,0.001041,0.000995,0.001228,0.001896,0.003294,0.001900,5.798073e-109,...,0.001409,0.001742,0.001248,0.006565,0.003079,0.002262,0.002107,0.001620,0.000772,0.001349
2021-03-10,0.001262,0.002431,NaN,0.001022,0.000985,0.001086,0.002363,0.002535,0.002146,2.899036e-109,...,0.001567,0.001900,0.001500,0.005892,0.003544,0.002190,0.002104,0.001938,0.000953,0.001057
2021-03-11,0.001493,0.003187,NaN,0.001733,0.001014,0.000879,0.002269,0.002541,0.005511,1.449518e-109,...,0.001745,0.002004,0.001608,0.006886,0.002861,0.002156,0.002137,0.002229,0.000999,0.001419


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-09      0.001500  0.000871  0.001563  0.000621  0.000291  0.000217   
2021-03-10      0.003026  0.001688  0.004042  0.000919  0.000545  0.001383   
2021-03-11      0.002216  0.001643  0.004335  0.001472  0.001391  0.001115   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-09      0.001136  0.001217  0.000282  0.000566  ... -0.000089   
2021-03-10      0.001063  0.001038  0.000578  0.000844  ... -0.000045   
2021-03-11      0.003746  0.001162  0.000289  0.000142  ...  0.000168   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-03-09     -0.000206  0.000411  0.000779   0.002607  0.001438  0.002472   
2021-03-10     -0.000103  0.000370  0.000390   0.001562  0.002044  0.001476   
2021-03-11     -0.000052  0.000679  0.000195   0.001947  0.003079  0.001458   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-03-09           -1.0  0.000322  0.000006  
2021-03-10           -1.0  0.000161  0.000003  
2021-03-11           -1.0  0.000080  0.006422  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-03-09,0.002274,0.000326,NaN,0.000479,0.003204,0.004265,0.000763,0.001695,0.004286,NaN,...,0.001846,0.003273,0.003847,0.007583,3.104409e-10,0.010197,0.002652,0.000612,0.002228,0.006663
2021-03-10,0.002904,0.000163,NaN,0.002629,0.003936,0.004531,0.000882,0.001687,0.005517,NaN,...,0.001699,0.003984,0.002426,0.003792,1.552204e-10,0.008112,0.003591,0.001166,0.002235,0.003331
2021-03-11,0.003996,0.000081,NaN,0.003052,0.004569,0.003869,0.001940,0.001424,0.003429,NaN,...,0.001582,0.004296,0.006986,0.001896,7.761021e-11,0.006746,0.002482,0.002300,0.001817,0.001666


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-03-09      0.000186  0.002007  0.000003 -0.001039  0.000254  0.027787   
2021-03-10      0.005528  0.006176  0.000001 -0.000520  0.004033  0.027051   
2021-03-11      0.013517  0.004794  0.009805 -0.000260  0.002017  0.013526   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-03-09      0.000008  0.000847  0.004557  0.013406  ...  0.037187   
2021-03-10      0.000004  0.000424  0.006743  0.018898  ...  0.018594   
2021-03-11      0.000002  0.001901  0.003371  0.009449  ...  0.009297   

Province_State                                                                 \
Admin2                Platte  Sheridan      Sublette Sweetwater         Teton   
2021-03-09      1.146738e-14  0.000434  9.373832e-21   0.000338  2.992303e-08   
2021-03-10      5.733689e-15  0.000217  4.686916e-21   0.000169  1.496152e-08   
2021-03-11      2.866844e-15  0.000109  2.343458e-21   0.000085  7.480758e-09   

Province_State                                                       
Admin2                 Uinta Unassigned      Washakie        Weston  
2021-03-09      1.520670e-15  -0.501961  5.824120e-13  1.236913e-11  
2021-03-10      7.603352e-16  -0.501961  2.912060e-13  6.184564e-12  
2021-03-11      3.801676e-16  -0.501961  1.456030e-13  3.092282e-12  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
